In [2]:
# in case of new notebook, use the commented code to load the data:

import pandas as pd

file1 = pd.read_csv('./file1.csv')
file2 = pd.read_csv('./file2.txt', sep = '\t')
file3 = pd.read_excel('./file3.xlsx')
file4 = pd.read_excel('./file4.xlsx')
column_names = file1.columns
#donors = pd.DataFrame(columns=column_names)
donors = pd.concat([file1,file2,file3,file4], axis=0)
cols = []
for colname in donors.columns:
    cols.append(colname.lower())
donors.columns = cols
donors = donors.rename(columns={'controln':'id',
                                'hv1':'median_home_val', 
                                'ic1':'median_household_income'})
donors['median_home_val'] =  pd.to_numeric(donors['median_home_val'], errors='coerce')
donors['ic5'] =  pd.to_numeric(donors['ic5'], errors='coerce')
donors = donors.drop_duplicates()
donors = donors.reset_index(drop=True)

donors.shape


(4001, 17)

In [3]:
donors.head()

,id,state,gender,median_home_val,median_household_income,ic4,hvp1,ic5,pobc1,pobc2,ic2,ic3,avggift,tcode,dob,domain,target_d
0,44060,FL,M,NaN,392,520.0,7,21975.0,6,16,430.0,466,28.000000,1,1901,C2,100.0
1,96093,IL,M,537.0,365,473.0,0,19387.0,1,89,415.0,410,5.666667,0,0,T2,7.0
2,43333,FL,F,725.0,301,436.0,3,18837.0,11,17,340.0,361,4.111111,0,2501,C2,5.0
3,21885,NC,M,NaN,401,413.0,7,14014.0,1,74,407.0,399,27.277778,0,2208,T2,38.0
4,190108,FL,F,995.0,252,348.0,0,17991.0,5,6,280.0,316,6.000000,28,0,C2,5.0


In [4]:
donors.dtypes
#donors.head(25)

id                           int64
state                       object
gender                      object
median_home_val            float64
median_household_income      int64
ic4                        float64
hvp1                         int64
ic5                        float64
pobc1                        int64
pobc2                        int64
ic2                        float64
ic3                          int64
avggift                    float64
tcode                        int64
dob                          int64
domain                      object
target_d                   float64
dtype: object

In [6]:
donors.isna().sum()

id                           0
state                        0
gender                     133
median_home_val             10
median_household_income      0
ic4                          1
hvp1                         0
ic5                          6
pobc1                        0
pobc2                        0
ic2                          1
ic3                          0
avggift                      0
tcode                        0
dob                          0
domain                       0
target_d                     0
dtype: int64

In [10]:

round(donors.isna().sum()/len(donors),4)*100  # shows the percentage of null values in a column
nulls_df = pd.DataFrame(round(donors.isna().sum()/len(donors),4)*100)
nulls_df
nulls_df = nulls_df.reset_index()
nulls_df
nulls_df.columns = ['header_name', 'percent_nulls']
nulls_df

,header_name,percent_nulls
0,id,0.00
1,state,0.00
2,gender,3.32
3,median_home_val,0.25
4,median_household_income,0.00
5,ic4,0.02
6,hvp1,0.00
7,ic5,0.15
8,pobc1,0.00
9,pobc2,0.00


In [11]:
columns_drop = nulls_df[nulls_df['percent_nulls']>3]['header_name']  # dummy case with 3
print(columns_drop.values)
#donors = donors.drop(columns_drop, axis=1)  # drop a list of columns DO NOT RUN THIS
#donors = donors.drop(['gender'], axis=1)  # drop a single column DO NOT RUN THIS

['gender']


In [13]:
# Replacing/imputing null values

donors[donors['median_home_val'].isna()==True].head(60) # checking rows that are null based on a specific column


,id,state,gender,median_home_val,median_household_income,ic4,hvp1,ic5,pobc1,pobc2,ic2,ic3,avggift,tcode,dob,domain,target_d
0,44060,FL,M,NaN,392,520.0,7,21975.0,6,16,430.0,466,28.000000,1,1901,C2,100.0
3,21885,NC,M,NaN,401,413.0,7,14014.0,1,74,407.0,399,27.277778,0,2208,T2,38.0
8,173223,CA,F,NaN,184,250.0,0,8708.0,2,63,206.0,235,8.818182,0,1801,T2,10.0
9,157988,CA,F,NaN,593,617.0,61,17838.0,7,62,NaN,619,6.666667,28,0,S1,10.0
35,116894,TX,M,NaN,368,531.0,5,33797.0,2,42,457.0,450,12.250000,1,2201,U1,20.0
3000,186307,CA,M,NaN,345,438.0,3,15915.0,2,58,388.0,417,16.142857,1,2511,C2,20.0
3005,150799,CA,M,NaN,257,393.0,8,10990.0,19,51,275.0,376,7.625000,1,0,U3,15.0
3014,38392,FL,F,NaN,408,464.0,11,13050.0,67,26,415.0,452,20.000000,0,3401,U2,20.0
3174,171205,CA,F,NaN,205,388.0,4,22845.0,4,71,221.0,337,7.454545,228,0,R2,12.0
3201,190408,FL,NaN,NaN,255,331.0,0,13727.0,13,7,273.0,294,6.600000,0,3104,C2,7.0


In [14]:
# strategy: drop rows that have null values (only if there are very few)
donors = donors[donors['ic2'].isna()==False] # Since these nulls are not a lot, we can filter them

donors.isna().sum()
# donors.shape

id                           0
state                        0
gender                     133
median_home_val              9
median_household_income      0
ic4                          1
hvp1                         0
ic5                          6
pobc1                        0
pobc2                        0
ic2                          0
ic3                          0
avggift                      0
tcode                        0
dob                          0
domain                       0
target_d                     0
dtype: int64

In [16]:
# strategy: impute a value for the missing value (fill in a value that we choose or calculate)
# import numpy
import numpy as np
mean_median_home_value = donors['median_home_val'].mean()
mean_median_home_value
donors['median_home_val'] = donors['median_home_val'].fillna(mean_median_home_value)

In [17]:
donors['median_home_val'].mean()

1157.3292407917818

In [18]:
donors.isna().sum()

id                           0
state                        0
gender                     133
median_home_val              0
median_household_income      0
ic4                          1
hvp1                         0
ic5                          6
pobc1                        0
pobc2                        0
ic2                          0
ic3                          0
avggift                      0
tcode                        0
dob                          0
domain                       0
target_d                     0
dtype: int64

In [ ]:
#Activity 1

Refer to the file files_for_activities/file1.csv for this exercise.

Load data (file1.csv) in a new Jupyter notebook.
Write the code to clean the columns 'ic4' and 'ic5' of null values in the dataframe.
Use the head() to check the new dataframe.

In [ ]:
#End of Activity 1

In [23]:
# Replacing null values for categorical variables
donors['gender'].unique()
donors['gender'].value_counts()

donors['gender'].value_counts(dropna=False)
#len(donors[donors['gender'].isna()==True])  # number of missing values


F          1954
M          1466
NaN         133
male        126
female      106
Female       75
U            68
Male         33
J            23
feamale      15
A             1
Name: gender, dtype: int64

In [24]:
# use most common value to fillna
donors['gender'] = donors['gender'].fillna('F')
len(donors[donors['gender'].isna()==True]) # now this number is 0



0

In [25]:
# Exporting this processed data to a csv
donors.to_csv('merged_clean_ver1.csv') # you can find this file inside files_for_lesson_and_activities folder

In [26]:
donors['gender'].value_counts(dropna=False)

F          2087
M          1466
male        126
female      106
Female       75
U            68
Male         33
J            23
feamale      15
A             1
Name: gender, dtype: int64

In [ ]:
#Activity 2


In [ ]:
# Refer to the file files_for_activities/merged_clean_ver1.csv for this exercise.

#1 Import the data from merged_clean_ver1.csv as a dataframe. There would be a column with the sequence of numbers (to the left of column id). Drop that column(s).


In [ ]:
# 2 Check the column state for null values. Replace those null values with the state that is represented largest number of times in that column


In [ ]:
#End of Activity 2


In [ ]:
donors.columns

In [27]:
# map functions
# list(map(len, donors.columns))
# list(map(lambda el: el.upper(), donors.columns)) # we did this with a for loop earlier
# # # donors
donors['gender'].unique() # check the unique values in the column
donors['gender'] = list(map(lambda x: x.upper(), donors['gender']))
donors['gender'].unique()

array(['M', 'F', 'FEMALE', 'MALE', 'U', 'J', 'FEAMALE', 'A'], dtype=object)

In [28]:
#donors['gender'].unique()  # check the unique elements in the column
# Now define a function to clean the column
def clean(x):
    if x in ['M', 'MALE']:
        return 'Male'
    elif x.startswith('F'):
        return 'Female'
    else:
        return 'U'

donors['gender'] = list(map(clean, donors['gender']))
donors['gender'].unique()  # To check the results again




array(['Male', 'Female', 'U'], dtype=object)

In [29]:
donors['gender'].value_counts()

Female    2283
Male      1625
U           92
Name: gender, dtype: int64

In [ ]:
# Examples of working with datetime format:

file = pd.read_csv('df_final_web_data_pt_1.csv')
file.head()

file['date_time'] = pd.to_datetime(file['date_time'], errors='coerce')
file.head()



In [ ]:
file['date_time'][0]

In [ ]:
file['date_time'][0].day
file['date_time'][0].month
file['date_time'][0].year
file['date_time'][0].isoweekday()  # Returns 1 for Monday and so on

file['date_time'][0].time()
file['date_time'][0].isoweekday()
file['date_time'][0].isoformat()
file['date_time'][0].strftime(format='%d-%m-%Y')
file['date_time'][0].strftime(format="%A %d. %B %Y")



In [ ]:
import time
from datetime import date

today = date.today()
today.strftime(format='%d-%m-%Y')

time.localtime(time.time())
time.gmtime(time.time())

In [ ]:
# Examples of working with string functions

string = " I am learning  data  analysis at Ironhack  . It is  super easy "
string.lower()
string.upper()
'34'.isdigit() # does not work with decimal numbers
str2 = '42'
str2.isdigit()
string = string.lstrip()
string
string.rstrip()
string.split()
string.split('.')
string.replace('  ', ' ')

In [ ]:
#Acticity 4
# Create a user-defined method to clean the column state in the dataframe.
# Use string functions to standardize the states to uppercase and use the strip function to clean the strings as well.
data['state'].unique()

def clean(x):
    x = x.upper()
    x = x.strip()
    if x in ['AZ', 'ARIZONA']:
        return 'AZ'
    elif x in ['CA', 'CALIFORNIA', 'CALI']:
        return 'CA'
    elif x in ['TN', 'TENNESSEE']:
        return 'TN'
    else:
        return x




In [ ]:
data['state'] = list(map(clean,data['state']))

data['state'].unique()

In [ ]:
#End Activity 4

In [ ]:
# Exporting this processed data to a csv
donors.to_csv('merged_clean_ver2.csv') 